# sn14-query-demonstration-helper
This notebook is meant to be used as an example to demonstrate how the subnet miners can be queried and how the validation process works. The implementation is slightly different from the normal implementation in the validators, since the logic is not 1:1 transferrable into a Jupyter Notebook. The changes are outlined in the code.

The first parts of the demonstrations are more about generic usage, while the second part answers to the questions within the subnet demo notebook (https://github.com/opentensor/subnet_demo_stack/blob/main/demo_template.ipynb).

Any issues with the demo notebook, please reach out directly to @ceterum_ or @m4k1_kun via Discord.

In [ ]:
# Import the necessary modules to demonstrate the validator
import bittensor as bt
from llm_defender.base.protocol importLLMDefender.SubnetProtocol
from llm_defender.core.validators.validator import SubnetValidator
from argparse import ArgumentParser
from llm_defender.base.utils import sign_data
from torch import argmax, argmin, nonzero, masked_select
from uuid import uuid4
from json import dumps
from secrets import token_hex
from time import time
from sys import exit
import requests
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from datasets import load_dataset

In [ ]:
# Setup subnet-related objects required to query the network

# Subnet parameters
parser = ArgumentParser()
parser.add_argument(
    "--alpha",
    default=0.9,
    type=float,
    help="The weight moving average scoring.",
)
parser.add_argument("--netuid", type=int, default=14, help="The chain subnet uid.")

# Define the bittensor configuration parameters here
parser.add_argument("--wallet.name", type=str, default="default")
parser.add_argument("--wallet.hotkey", type=str, default="default")
parser.add_argument("--subtensor.network", type=str, default="finney")


# Setup Subnet validator
subnet_validator = SubnetValidator(parser=parser)

# Apply configuration
subnet_validator.apply_config(bt_classes=[bt.subtensor, bt.logging, bt.wallet])

# Initialize validator
wallet, subtensor, dendrite, metagraph = subnet_validator.setup_bittensor_objects(
    subnet_validator.neuron_config
)
subnet_validator.wallet = wallet
subnet_validator.subtensor = subtensor
subnet_validator.dendrite = dendrite
subnet_validator.metagraph = metagraph
subnet_validator.init_default_scores()
subnet_validator.remote_logging = False

In [ ]:
# Helper functions are needed resolve certain parameters before sending
# out the query. These are not part of the standard subnet
# implementation.

def get_dataset_entries(seed=42, samples=50):
    dataset = load_dataset("synapsecai/synthetic-prompt-injections")
    test_dataset = dataset["test"]

    dataset = test_dataset.shuffle(seed=seed)
    samples = test_dataset.select(range(samples))

    return samples

def get_top_miner() -> bt.NeuronInfo:
    """This function returns the NeuronInfo for the top performing miner
    based on incentive"""

    # Get incentives and the top performing UID
    incentives = subnet_validator.metagraph.I
    uid = argmax(incentives)
    print(f"Top UID: {uid} with incentive: {incentives[uid]}")

    # Return AxonInfo based on the UID
    neuron_info = subnet_validator.metagraph.neurons[uid]
    return neuron_info

def get_bottom_miner() -> bt.NeuronInfo:
    """This function returns the NeuronInfo for the top performing miner
    based on incentive"""

    # Get incentives and the top performing UID
    incentives = subnet_validator.metagraph.I
    non_zero_mask = incentives > 0.003 # limit to miners that are performing somewhat ok, there are couple of mining validators that do not response properly
    min_non_zero_value = masked_select(incentives, non_zero_mask).min()
    
    # Step 3: Get the index of the minimum non-zero value
    uid = (incentives == min_non_zero_value).nonzero(as_tuple=True)[0][0]
    
    print(f"Bottom UID: {uid} with incentive: {incentives[uid]}")

    # Return AxonInfo based on the UID
    neuron_info = subnet_validator.metagraph.neurons[uid]
    return neuron_info


def standard_response_processing(responses, queries, uuids, axon_to_query, no_output=False) -> list:
    """This functions performs the standard response processing and
    returns list of processed responses"""

    processed_responses = []
    for i, entry in enumerate(responses):
        if not no_output:
            print(
                f"Received non-empty response from the miner:\n{dumps(entry.output, indent=4)}"
            )

        # The process_response function is responsible for handling valid
        # responses. It is executed if any one of the responses is non-empty.
        response_data = subnet_validator.process_responses(
            query=queries[i],
            processed_uids=[axon_to_query.uid],
            responses=[entry],
            synapse_uuid=uuids[i],
        )

        processed_responses.append(response_data)

        for res in response_data:
            if subnet_validator.miner_responses:
                if res["hotkey"] in subnet_validator.miner_responses:
                    subnet_validator.miner_responses[res["hotkey"]].append(res)
                else:
                    subnet_validator.miner_responses[res["hotkey"]] = [res]
            else:
                subnet_validator.miner_responses = {}
                subnet_validator.miner_responses[res["hotkey"]] = [res]
    
    return processed_responses

def calculate_response_metrics(hotkey) -> dict:
    """This function calculates some standard metrics for the miner
    responses contained in the local memory"""

    data = subnet_validator.miner_responses[hotkey]
    
    labels = []
    classifications = []

    for i,entry in enumerate(data):
        if entry["target"] is None or "confidence" not in entry["response"].keys() or entry["response"]["confidence"] is None:
            continue
        labels.append(round(entry["target"]))
        classifications.append(round(entry["response"]["confidence"]))


    # Calculate metrics
    accuracy = accuracy_score(labels, classifications)
    precision = precision_score(labels, classifications)
    recall = recall_score(labels, classifications)
    f1 = f1_score(labels, classifications)

    res = {
        "hotkey": hotkey,
        "count": len(data),
        "metrics": {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1
        },
        "scoring_averages": {
            "total_score": sum(entry["scored_response"]["scores"]["total"] for entry in data) / len(data) if data else None,
            "distance_score": sum(entry["scored_response"]["scores"]["distance"] for entry in data) / len(data) if data else None,
            "speed_score": sum(entry["scored_response"]["scores"]["speed"] for entry in data) / len(data) if data else None,
            "raw_distance": sum(entry["scored_response"]["raw_scores"]["distance"] for entry in data) / len(data) if data else None,
            "raw_speed": sum(entry["scored_response"]["raw_scores"]["speed"] for entry in data) / len(data) if data else None,
            "distance_penalty": sum(entry["scored_response"]["penalties"]["distance"] for entry in data) / len(data) if data else None,
            "speed_penalty": sum(entry["scored_response"]["penalties"]["speed"] for entry in data) / len(data) if data else None
        }
    }

    return res

In [ ]:
# Setup variables that are needed throughout the execution
axon_to_query = get_top_miner()
print(f"Axon to query: {axon_to_query}")

In [ ]:
# This code block executes a normal query where the prompt is fetched
# through the Prompt API. This is the primary query executed by the
# validators.

# Get the query to send for the miner. The query contains the metadata
# associated with the prompt that is to be analyzed by the miners. In
# order to query the prompt API, the hotkey must be registered in the
# subnet 14 and have at least 20k staked TAO.

# Send queries to the miner
#
# DANGER: There is a rate-limit of 100 requests every 10 minutes. If you
# are executing this notebook from a host with the same IP with your
# actual validator, you are at risk of being rate-limited.
#
# Please use the dataset-based approach below for queries with multiple
# prompts. Using two prompts should be enough to be able to demonstrate
# how the process works. The dataset-based approach below has more detailed examples

async def normal_query(normal_axon, n=2):
    responses = []
    queries = []
    uuids = []
    for _ in range(0, n):
        # UUID is used to uniquely identify the request
        synapse_uuid = str(uuid4())
        uuids.append(synapse_uuid)

        # Miner hotkeys are used to by the Prompt API to create prompts for the
        # miners to fetch. Each prompt can be fetched only once.
        miner_hotkeys = [normal_axon.hotkey]

        # Executing the serve_prompt method fetches the query metadata from the Prompt API
        query = subnet_validator.serve_prompt(
            synapse_uuid=synapse_uuid, miner_hotkeys=miner_hotkeys
        )
        queries.append(query)

        print(f"Query metadata:\n{dumps(query, indent=4)}")

        # Prepare the query. The query is signed and includes a nonce, to prevent replay of validator requests
        nonce = token_hex(24)
        timestamp = str(int(time()))
        signature = LLMDefender.sign_data(
            hotkey=subnet_validator.wallet.hotkey,
            data=f"{synapse_uuid}{nonce}{subnet_validator.wallet.hotkey.ss58_address}{timestamp}",
        )
        miner_res = await subnet_validator.dendrite(
            normal_axon.axon_info,
           LLMDefender.SubnetProtocol(
                analyzer=query["analyzer"],
                subnet_version=subnet_validator.subnet_version,
                synapse_uuid=synapse_uuid,
                synapse_signature=signature,
                synapse_nonce=nonce,
                synapse_timestamp=timestamp,
            ),
            timeout=6,
            deserialize=True,
        )
        responses.append(miner_res)
    
    return responses, queries, uuids

miner_responses, queries, uuids = await normal_query(normal_axon=axon_to_query)

In [ ]:
def process_miner_responses(miner_responses, queries, uuids, axon_to_process, no_output=False):
    # This code block executes the built-in process-method for the responses
    # retrieved by the previous code block.

    # If all of the responses in the list of returned responses are empty
    # the validator would normally set the score for the queries to 0.0.
    if not miner_responses or all(item.output is None for item in miner_responses):
        print(f"Received empty responses: {miner_responses}")
        exit(-1)

    processed_responses = standard_response_processing(miner_responses, queries, uuids, axon_to_process, no_output)

    # After processing the responses, the validator would normally update
    # its local knowledge of the miner responses. The local knowledge of the
    # miner responses are used within the reward mechanism.

    if not no_output:
        print(f"Processed response:\n{dumps(processed_responses, indent=4)}")

In [ ]:
# This code block starts the execution of the non-standard way of
# querying the miners. This code block creates prompts for the miners to
# analyze rather than using the synthetic prompts generated by the
# prompt api. This could be considered as an early prototype of the
# subnet-api the subnet owners are building.


def get_subnet_api(synapse_uuid, data) -> dict:
    """Retrieves a prompt from the prompt API"""

    nonce = token_hex(24)
    timestamp = str(int(time()))
    signature = LLMDefender.sign_data(
        hotkey=subnet_validator.wallet.hotkey,
        data=f"{synapse_uuid}{nonce}{timestamp}",
    )

    headers = {
        "X-Hotkey": subnet_validator.wallet.hotkey.ss58_address,
        "X-Signature": signature,
        "X-SynapseUUID": synapse_uuid,
        "X-Timestamp": timestamp,
        "X-Nonce": nonce,
        "X-Version": str(40),
        # "X-Version": str(subnet_validator.subnet_version),
        "X-API-Key": subnet_validator.wallet.hotkey.ss58_address,
    }

    prompt_api_url = "https://subnet-api.synapsec.ai/subnet"
    try:
        # get prompt
        res = requests.post(
            url=prompt_api_url, headers=headers, data=dumps(data), timeout=12
        )
        # check for correct status code
        if res.status_code == 201:
            # get prompt entry from the API output
            prompt_entry = res.json()
            # check to make sure prompt is valid
            bt.logging.trace(f"Loaded remote prompt to serve to miners: {prompt_entry}")
            return prompt_entry

        else:
            bt.logging.warning(
                f"Unable to get prompt from the Prompt API: HTTP/{res.status_code} - {res.json()}"
            )
    except requests.exceptions.ReadTimeout as e:
        print(e)
        bt.logging.error(f"Prompt API request timed out: {e}")
    except requests.exceptions.JSONDecodeError as e:
        print(e)
        bt.logging.error(f"Unable to read the response from the prompt API: {e}")
    except requests.exceptions.ConnectionError as e:
        print(e)
        bt.logging.error(f"Unable to connect to the prompt API: {e}")
    except Exception as e:
        print(e)
        bt.logging.error(f"Generic error during request: {e}")

In [ ]:
# Setup prompts and labels to be queried. The dataset is used as a
# reference contains prompts that have already been served to the
# network, so it is likely a top miner is pretty good at classifying the
# prompts.

async def subnet_api_demo_query(demo_axon, no_output=False):
    samples = get_dataset_entries()
    
    # Now, separate the text and prompts into two lists
    prompts = [sample['text'] for sample in samples]
    labels = [1 if sample['label'] == 'malicious' else 0 for sample in samples]

    # Store miner responses and queries in a list
    responses = []
    queries = []
    uuids = []

    # Send queries to miners
    for i,_ in enumerate(prompts):
        
        # UUID is used to uniquely identify the request
        synapse_uuid = str(uuid4())
        uuids.append(synapse_uuid)
        
        # Prepare data object to push to Prompt API
        data = {
            "hotkey": demo_axon.hotkey,
            "prompt": prompts[i],
            "label": str(labels[i]),
            "analyzer": "Prompt Injection",
        }
        
        # Push prompt to prompt API and get the query metadata
        query = get_subnet_api(synapse_uuid=synapse_uuid, data=data)
        queries.append(query)

        if not no_output:
            print(f"Query metadata:\n{dumps(query, indent=4)}")

        # Prepare the query. The query is signed and includes a nonce, to prevent replay of validator requests
        nonce = token_hex(24)
        timestamp = str(int(time()))
        signature = LLMDefender.sign_data(
            hotkey=subnet_validator.wallet.hotkey,
            data=f"{synapse_uuid}{nonce}{subnet_validator.wallet.hotkey.ss58_address}{timestamp}",
        )

        miner_res = await subnet_validator.dendrite(
            demo_axon.axon_info,
           LLMDefender.SubnetProtocol(
                analyzer=query["analyzer"],
                subnet_version=subnet_validator.subnet_version,
                synapse_uuid=synapse_uuid,
                synapse_signature=signature,
                synapse_nonce=nonce,
                synapse_timestamp=timestamp,
            ),
            timeout=6,
            deserialize=True,
        )
        responses.append(miner_res)

    return responses, queries, uuids
    
miner_responses, queries, uuids = await subnet_api_demo_query(demo_axon=axon_to_query)

In [ ]:
process_miner_responses(miner_responses, queries, uuids, axon_to_query)

In [ ]:
# Print miner metrics from the stored miner responses
metrics = calculate_response_metrics(hotkey=axon_to_query.hotkey)
print(dumps(metrics,indent=4))

(A) Demonstrate the quality of the communication between miners and validators coming from the top miner.

In [ ]:
# The communication between miner and validator can be demonstrated by executing the normal_query() function. This is a representative of the normal behavior within the subnet.
# There are four prints in this cell block. The "Query Metadata" is the metadata the validator uses to score the miner response. 
# The second print contains the actual response from the miner. 
# The third one displays the scored response (raw_distance and raw_speed are the key fields to look at: raw_distance equals to absolute distance to correct label and the raw_speed is calculated based on the response speed (response_speed/max_timeout))

# Get the top miner
top_miner = get_top_miner()

# Query the miner
miner_responses, queries, uuids = await normal_query(normal_axon=top_miner, n=1)

# Process the response from the miner
process_miner_responses(miner_responses, queries, uuids, top_miner)


(B) Justify the difference in incentive for miners in different tiers (eg.quantile 1 VS quantile 3).

In [ ]:
# Our scoring is based purely on the score calculated from the raw_distance and raw_speed. There are no performance buckets, but the difference in the scoring is determined by the response quality, effectively the average accuracy.
# This block uses the subnet_api_demo_query() function to get top and bottom miner to process 50 prompts and displays 
# Bulk-querying is not yet supported by the miners, so sending out the prompts and processing them can take 60-90 seconds.

# Before we proceed, we must clear our local knowledge of the miner replies to not cause deviation in the score calculations
subnet_validator.miner_responses = None

# Get the top miner
top_miner = get_top_miner()
bottom_miner = get_bottom_miner()

# Query the top miner
top_miner_responses, top_queries, top_uuids = await subnet_api_demo_query(demo_axon=top_miner, no_output=True)

# Process the response from the top miner
process_miner_responses(top_miner_responses, top_queries, top_uuids, top_miner, True)

# Query the bottom miner
bottom_miner_responses, bottom_queries, bottom_uuids = await subnet_api_demo_query(demo_axon=bottom_miner, no_output=True)

# Process the response from the bottom miner
process_miner_responses(bottom_miner_responses, bottom_queries, bottom_uuids, bottom_miner, True)

# Print metrics
# --Note that when using the dataset in huggingface, there is a high possibility that all of the metrics equal to 1

In [ ]:
# Print miner metrics from the stored miner responses for top miner
metrics = calculate_response_metrics(hotkey=top_miner.hotkey)
print(dumps(metrics,indent=4))

# Print miner metrics from the stored miner responses for bottom miner
metrics = calculate_response_metrics(hotkey=bottom_miner.hotkey)
print(dumps(metrics,indent=4))

(C) (If applicable) Show the landscape and variety of miners.

In [ ]:
# This is not relevant for our subnet. This is somewhat demonstrated by the cell for (B)

(D) (If applicable) Demonstrate the effectiveness of the scoring mechanism.

In [ ]:
# This is also covered by the cell for (B) 

(E) (If applicable) Show the dataset that was used by the validator.

In [ ]:
# The validator uses a mix of dataset-based prompts and synthetic prompts that are generated out-of-bounds and pushed to an SQS queue used by the prompt API.
# The prompt API issues prompts to validators by using 50% dataset and 50% unique synthetic prompts (which have not been seen before sending out to the miners).
# Correctly classifying a prompt from dataset is given weight of 10% (0.1 multiplier on score change) and unique prompts are given weight of 100% (1.0 multiplier in scoring)
# Datasets that are rotated ouf of the prompt API are published in Huggingface to allow miners to easier fine-tune their models based on formerly analyzed data. 

samples = get_dataset_entries()
print(dumps(samples[:10],indent=4))

: 

(F) (If applicable) Show the use of any API and/or links to a frontend.

In [ ]:
# The APIs used by the subnet are demonstrated within the previous cells of this notebook. The subnet has three APIs:
# - Prompt API 
#   - Used to push synthetically generated and dataset-based prompts to validators (GPT4)
#   - Temporary solution pending for the development of robust solution to generate synthetic data on the validators, most likely by utilizing other subnets to generate the data. 
#   - There are both ethical concerns and technical issues we have not yet managed to address.
# - Fetch API 
#   - Used by the miners to fetch the prompts 
#   - Temporary solution to prevent relay attacks
# - Subnet API
#   - Used by validators to push prompts into the fetch API
#   - To be published as open source Subnet API any validator can run (once fetch API has been replaced with a solution that can be run on the validator)
# These are documented at: https://docs.synapsec.ai
# Code can be shared upon request, but we are not yet ready to share these to everyone